In [3]:
from stat import S_ISREG, ST_CTIME, ST_MODE, ST_MTIME, ST_GID
import os, sys, time
from datetime import datetime
import json
import subprocess
from configparser import ConfigParser

In [4]:
class VerzovaciAutomat:
    def __init__(self):
        pass

    def denSouboruVAdresari(self, cestaKAdresari):
        ''' Funkce ktera vrátí datum '''
        self.cestaKAdresari = cestaKAdresari
        #  získame všechny soubory v adresáři s/ statistikami (fn jsou jednotlive soubory)
        seznamSouboruKeKontrole = [os.path.join(self.cestaKAdresari, fn) for fn in os.listdir(self.cestaKAdresari)]

        seznamStatCesta = [(os.stat(path), path) for path in seznamSouboruKeKontrole]
        # ponechat pouze běžné soubory, vložit datum vytvoření ((stat[ST_MTIME] - vlozi datum posledni zmeny)
        serazeneSoubory = ((stat[ST_MTIME], path) for stat, path in seznamStatCesta if S_ISREG(stat[ST_MODE])) 
        # smycka ktera vrati posledni modifikovny soubor
        for cdate, path in sorted(serazeneSoubory, reverse=True):
#             datumSouboru = time.ctime(cdate)
            ts = datetime.fromtimestamp(cdate).strftime('%Y-%m-%d')
#             datumSouboruDen = [int(s) for s in datumSouboru.split() if s.isdigit()][0]
#             print(f"datumSouboruDen {datumSouboruDen}")
            return ts

    def jsouDnesZmenyVSouborech(self, configCesty, uvodCesty):
        ''' Funkce která vrátí výsldný slovník kde je uvedeno jestli jsou dnes konkrétní zdroje upraveny nebo ne '''
        self.configCesty = configCesty
        self.uvodCesty = uvodCesty

        # prace s json soubory
        with open(self.configCesty) as json_file:

            self.configCesty = json.load(json_file)

        # vytvoreni prazdnych slovniku pro zaznamenani posledni zmeny souboru
        verzovaciAutomat = VerzovaciAutomat()
        seznamVsechZmenOstre = []
        seznamVsechZmenStare = []
        seznamVsechZmenFreeze = []
        # smycka pres vsechny adresare ktere kontrolujeme
        for zdroje, slovnikVyvoj in self.configCesty.items():
            if zdroje == 'Ostre':
                for vyvoj, cestaKonec in slovnikVyvoj.items():
                    # vytvořím seznam vsech zmen v ostrých zdrojích
                    cesta = self.uvodCesty + cestaKonec 
                    seznamVsechZmenOstre.append((verzovaciAutomat.denSouboruVAdresari(cesta)))              
            if zdroje == 'Stare':                                                                                        
                for vyvoj, cestaKonec in slovnikVyvoj.items():
                    # vytvořím seznam vsech zmen v starých zdrojích
                    cesta = self.uvodCesty + cestaKonec
                    seznamVsechZmenStare.append(verzovaciAutomat.denSouboruVAdresari(cesta))              
            if zdroje == 'Freeze':
                for vyvoj, cestaKonec in slovnikVyvoj.items():
                    # vytvořím seznam vsech zmen v freeze zdrojích
                    cesta = self.uvodCesty + cestaKonec
                    seznamVsechZmenFreeze.append(verzovaciAutomat.denSouboruVAdresari(cesta))
        # vytvoření slovníku se zmenami ve všech zdrojich
        seznamVsechZmen = {"Ostre" : seznamVsechZmenOstre, "Stare" : seznamVsechZmenStare, "Freeze" : seznamVsechZmenFreeze}
        nowDate = datetime.now()  # aktuální datum a čas
        datumDnesniDen = nowDate.strftime('%Y-%m-%d') # 
        print(f"Dnesni datum je: {datumDnesniDen}")
        print()
        for z, zmeny in seznamVsechZmen.items():
            print(f'Datum posledni zmeny v {z}: {max(zmeny)}')
            print()

        # vytvoření slovníku s konečnými výsledky podle kterých se rozhodujeme zda verze zvedat nebo ne 
        konecneVysledky = {}
        if datumDnesniDen == max(seznamVsechZmenOstre):
            konecneVysledky['Ostre'] = True
        else:
            konecneVysledky['Ostre'] = False

        if datumDnesniDen == max(seznamVsechZmenStare):
            konecneVysledky['Stare'] = True
        else:
            konecneVysledky['Stare'] = False

        if datumDnesniDen == max(seznamVsechZmenFreeze):
            konecneVysledky['Freeze'] = True
        else:
            konecneVysledky['Freeze'] = False
        
        # vytvoření slovníku pro tisknutí výsledků
        konecneVysledkyAnoNe = {}
        for z, anone in konecneVysledky.items():
            if z == 'Ostre':
                if anone == True:
                    konecneVysledkyAnoNe[z] = "Ano"
                else:
                    konecneVysledkyAnoNe[z] = "Ne"
            elif z == 'Stare':
                if anone == True:
                    konecneVysledkyAnoNe[z] = "Ano"
                else:
                    konecneVysledkyAnoNe[z] = "Ne"
            else:
                if anone == True:
                    konecneVysledkyAnoNe[z] = "Ano"
                else:
                    konecneVysledkyAnoNe[z] = "Ne"

        print(f'Byly dnes zmeneny zdroje: {konecneVysledkyAnoNe}')
        print()
        return konecneVysledky


    def aktualniVerzeSVN(self, soubor):
        ''' Funkce která vrátí aktuální číslo verze z SVN'''
        self.soubor = soubor
        with open(self.soubor, 'r') as file:
            # načteme seznam řádků do data
            data = file.readlines()    
        # nyní vybereme 2. a 4. řádek
        aktualniVerzeSVN = []
        radky = [1,3]
        for i in radky:
            # nacteni konkretniho radku
            heoCisloVerze = data[i]
            # posledni 4 cisla
            posledni4Cisla = int(heoCisloVerze[-5:])
            # převedeme intiger na string a přidáme nuly na začátek řetězce podle uvedeného délky
            posledni4Cisla = str(posledni4Cisla).zfill(4)
            # přidání do seznamu
            aktualniVerzeSVN.append(posledni4Cisla)

        return aktualniVerzeSVN


    def ziskaniCtyrCisli(self, soubor):
        ''' Funkce vrátí již zvednuté číslo verze'''
        self.soubor = soubor
        with open(self.soubor, 'r') as file:
            # načteme seznam řádků do data
            data = file.readlines()    
        # nyní změnime 2. a 4. řádek, všineme si, že musíme přidat nový řádek /n znamená že odskočíme na další řádku
        radky = [1,3]
        for i in radky:
            # nacteni konkretniho radku
            heoCisloVerze = data[i]    
            # verze bez posledniho 4 cisli
            zacatekCislaVerze = heoCisloVerze[:-5]
            # posledni 4 cisla
            posledni4CislaInt = int(heoCisloVerze[-5:])
            # převedeme intiger na string a přidáme nuly na začátek řetězce podle uvedeného délky
            posledni4Cisla = str(posledni4CislaInt).zfill(4)
            return posledni4Cisla  

    def zvyseniVerze(self, soubor):
        ''' Funkce zvýší číslo verze a uloží ho do souboru '''
        self.soubor = soubor
        with open(self.soubor, 'r') as file:
            # načteme seznam řádků do data
            data = file.readlines()    
        # nyní změnime 2. a 4. řádek, všineme si, že musíme přidat nový řádek /n znamená že odskočíme na další řádku
        radky = [1,3]
        for i in radky:
            # nacteni konkretniho radku
            heoCisloVerze = data[i]    
            # verze bez posledniho 4 cisli
            zacatekCislaVerze = heoCisloVerze[:-5]
            # posledni 4 cisla
            posledni4Cisla = int(heoCisloVerze[-5:])
            # převedeme intiger na string a přidáme nuly na začátek řetězce podle uvedeného délky
            zvyseny4Cisla = str(posledni4Cisla + 1).zfill(4)
            # vytvořeni výsledného stringu
            vyslednyStringVerze = zacatekCislaVerze + zvyseny4Cisla + '\n'
            # uložíme výsledné stringy na konkrétní řádky
            data[i] = vyslednyStringVerze
        # uložení zpět do konkrétního souboru
        with open(self.soubor, 'w') as file:
            file.writelines(data)      

    def upraveniCommitSouboru(self, souborCommitSVN, zdroje, message, uvodCesty):
        ''' Funkce která upraví commit soubor do výsledné podoby podle konkrétních zdrojů '''
        self.souborCommitSVN = souborCommitSVN
        self.zdroje = zdroje
        self.message = message
        self.uvodCesty = uvodCesty

        with open(self.souborCommitSVN, 'r') as file:
            # načteme seznam řádků do data
            data = file.readlines()    
        # nyní změnime 14. řádek
        radek = 13    
        if self.zdroje == 'Ostre':        
            # nacteni konkretniho radku (v našem případě 14 řádek)
            heoCisloVerze = data[radek]
            cestaNaSvn = '"c:\\Program Files\\TortoiseSVN\\bin\\svn.exe"'
            commit = 'commit' 
            konecCesty = '\\Ostre\\asseco\\_Verze - kopie.inc -m'
            celaCesta = self.uvodCesty + konecCesty
            msg = f'"{self.message}"'
            # Vytvoření výsledné zprávy do commit souboru, který vložíme na konkrétní řádek
            x = cestaNaSvn + ' ' + commit + ' ' + celaCesta + ' ' + msg + '\n'
            # uložíme výsledné stringy na konkrétní řádky
            data[radek] = x
        elif self.zdroje == 'Stare':        
            # nacteni konkretniho radku
            heoCisloVerze = data[radek]
            cestaNaSvn = '"c:\\Program Files\\TortoiseSVN\\bin\\svn.exe"'
            commit = 'commit' 
            konecCesty = '\\Stare\\asseco\\_Verze - kopie.inc -m'
            celaCesta = self.uvodCesty + konecCesty
            msg = f'"{self.message}"'
            x = cestaNaSvn + ' ' + commit + ' ' + celaCesta + ' ' + msg + '\n'
            data[radek] = x
        else:        
            # nacteni konkretniho radku
            heoCisloVerze = data[radek]
            cestaNaSvn = '"c:\\Program Files\\TortoiseSVN\\bin\\svn.exe"'
            commit = 'commit' 
            konecCesty = '\\Freeze\\asseco\\_Verze - kopie.inc -m'
            celaCesta = self.uvodCesty + konecCesty
            msg = f'"{self.message}"'
            x = cestaNaSvn + ' ' + commit + ' ' + celaCesta + ' ' + msg + '\n'
            data[radek] = x

        with open(self.souborCommitSVN, 'w') as file:
            file.writelines(data) 

    def updateFromSVN(self, zdroje):
        ''' funkce která načte konkretní bat soubor a pomocí něho provedeme update z SVN '''
        self.zdroje = zdroje
        if self.zdroje == 'Ostre':
           subprocess.call([r'C:\Users\zdenek.ptak\Repository\CisloVerze\updateSvnOstre.bat'])
        elif self.zdroje == 'Stare':
           subprocess.call([r'C:\Users\zdenek.ptak\Repository\CisloVerze\updateSvnStare.bat'])
        else:
           subprocess.call([r'C:\Users\zdenek.ptak\Repository\CisloVerze\updateSvnFreeze.bat'])


    def commitToSVN(self):
        ''' funkce která načte konkretní bat soubor a pomocí něho provedeme commit na SVN '''
        subprocess.call([r'C:\Users\zdenek.ptak\Repository\CisloVerze\commitSvn.bat'])

    def ziskaniVerzeZIni(self, iniSoubor, zdroje):
        ''' Funkce která načte poslední verzi která je uložena v INI souboru'''
        self.iniSoubor = iniSoubor
        self.zdroje = zdroje
        # instantiate
        config = ConfigParser()
        # načtení existujiciho souboru
        config.read(self.iniSoubor)
        # naštení hodnot z ini souboru podle konkretních zdrojů
        if self.zdroje == 'Ostre':
            heo2verze = config.get('HeO2_Ostre', 'verze')
            heo3verze = config.get('HeO3_Ostre', 'verze')
        elif self.zdroje == 'Stare':
            heo2verze = config.get('HeO2_Stare', 'verze')
            heo3verze = config.get('HeO3_Stare', 'verze')
        else:
            heo2verze = config.get('HeO2_Freeze', 'verze')
            heo3verze = config.get('HeO3_Freeze', 'verze')
        return [heo2verze, heo3verze]

    def ulozeniDoIni(self, iniSoubor, zdroje, noveCisloVerze):
        ''' Funkce která uloží nové čísla verzí do INI souboru '''
        self.iniSoubor = iniSoubor
        self.zdroje = zdroje
        self.noveCisloVerze = noveCisloVerze
        
        
        config = ConfigParser()
        config.read(self.iniSoubor)
        # zmena existujicich hodnot
        if self.zdroje == 'Ostre':
            config.set("HeO2_Ostre","verze", self.noveCisloVerze)
            config.set("HeO3_Ostre","verze", self.noveCisloVerze)
        elif self.zdroje == 'Stare':
            config.set("HeO2_Stare","verze", self.noveCisloVerze)
            config.set("HeO3_Stare","verze", self.noveCisloVerze)
        else:
            config.set("HeO2_Freeze","verze", self.noveCisloVerze)
            config.set("HeO3_Freeze","verze", self.noveCisloVerze)

        with open(self.iniSoubor, 'w') as configfile:
            config.write(configfile)

    def vyslednaZprava(self, cisloVerze):
        ''' Funkce která vytvoří zprávu pro commit na SVN '''
        self.cisloVerze = cisloVerze
        msg = f"Automatické zvýšení verze pro noční buildy: {self.cisloVerze}"
        return msg  
    
    def dokonceni(self, zdroje, cesta, iniSoubor, vysledkyPoslednichZmen, souborCommitSVN, uvodCesty):
        self.zdroje = zdroje
        self.cesta = cesta
        self.iniSoubor = iniSoubor
        self.vysledkyPoslednichZmen = vysledkyPoslednichZmen
        self.souborCommitSVN = souborCommitSVN
        self.uvodCesty = uvodCesty
        
        # vytvoření objektu verzovaciAutomat
        verzovaciAutomat = VerzovaciAutomat()
        # aktuální čísla verzí v SVN
        aktualniSVN = verzovaciAutomat.aktualniVerzeSVN(self.cesta)
        # aktuální čísla verzí v INI
        aktualniINI = verzovaciAutomat.ziskaniVerzeZIni(self.iniSoubor, self.zdroje)
        print(f'Aktualni cislo verze na SVN - HeO2: {aktualniSVN[0]} a HeO3: {aktualniSVN[1]}')
        print(f'Aktualni cislo verze v INI - HeO2: {aktualniINI[0]} a HeO3: {aktualniINI[1]}')
        
        # porovnání verzí v SVN a v INI podle kterého automaticky zvednem verzi nebo zůstane stejná
        if aktualniSVN[0] == aktualniINI[0] and aktualniSVN[1] == aktualniINI[1]:
            print(f"{self.zdroje}:  zdroje nebyly zvednuty manuálně")
            if self.vysledkyPoslednichZmen[self.zdroje] == True:
                print(f'{self.zdroje}:  do zdrojů byly vloženy úpravy')        
                verzovaciAutomat.zvyseniVerze(self.cesta)
                noveCisloVerze = verzovaciAutomat.ziskaniCtyrCisli(self.cesta)
                verzovaciAutomat.ulozeniDoIni(self.iniSoubor, self.zdroje, noveCisloVerze)
                message = verzovaciAutomat.vyslednaZprava(noveCisloVerze)
                print(f"Vyssledna zprava je: {message}")
                verzovaciAutomat.upraveniCommitSouboru(self.souborCommitSVN, self.zdroje, message, self.uvodCesty)
                print(f"{self.zdroje}:  zdroje byly zvyseny na {noveCisloVerze}")
                print()
#                 verzovaciAutomat.commitToSVN()
            else:
                print(f"{self.zdroje}:  zdroje nebyly upraveny")  
                print()
        else:
            print(f"{self.zdroje}:  zdroje byly zvýšeny manuálně")
            verzovaciAutomat.ulozeniDoIni(self.iniSoubor, self.zdroje, aktualniSVN[0])
            print(f'{self.zdroje}:  do ini souboru do bylo vlozeno cislo verze - {aktualniSVN[0]}')
            print()

In [27]:
verzovaciAutomat = VerzovaciAutomat()
souboryKeKontrole = 'C:/Users/zdenek.ptak/Repository/CisloVerze/souborykekontrole.json'
iniSoubor = 'posledniUlozenaVerze.ini'
souborCommitSVN = 'commitSvn.bat'
uvodCesty = "C:\\HeO_vyroba_distribuce\\Zdroje"

cestyKeZdrojum = {
    "Ostre": "\\Ostre\\asseco\\_Verze - kopie.inc",
    "Stare": "\\Stare\\asseco\\_Verze - kopie.inc", 
    "Freeze": "\\Freeze\\asseco\\_Verze - kopie.inc"
}
for zdroje, konecCesty in cestyKeZdrojum.items():
    # update z SVN
    verzovaciAutomat.updateFromSVN(zdroje)
    cestyKeZdrojum[zdroje] = uvodCesty + konecCesty

vysledkyPoslednichZmen = verzovaciAutomat.jsouDnesZmenyVSouborech(souboryKeKontrole, uvodCesty)

for zdroje, cesta in cestyKeZdrojum.items():
    print(cesta)
    if zdroje == 'Ostre':
        verzovaciAutomat.dokonceni(zdroje, cesta, iniSoubor, vysledkyPoslednichZmen, souborCommitSVN, uvodCesty)
    elif zdroje == 'Stare':
        verzovaciAutomat.dokonceni(zdroje, cesta, iniSoubor, vysledkyPoslednichZmen, souborCommitSVN, uvodCesty)       
    else:
        verzovaciAutomat.dokonceni(zdroje, cesta, iniSoubor, vysledkyPoslednichZmen, souborCommitSVN, uvodCesty)

Dnesni datum je: 2020-11-18

Datum posledni zmeny v Ostre: 2020-11-18

Datum posledni zmeny v Stare: 2020-11-18

Datum posledni zmeny v Freeze: 2020-11-13

Byly dnes zmeneny zdroje: {'Ostre': 'Ano', 'Stare': 'Ano', 'Freeze': 'Ne'}

C:\HeO_vyroba_distribuce\Zdroje\Ostre\asseco\_Verze - kopie.inc
Aktualni cislo verze na SVN - HeO2: 0999 a HeO3: 0999
Aktualni cislo verze v INI - HeO2: 0999 a HeO3: 0999
Ostre:  zdroje nebyly zvednuty manuálně
Ostre:  do zdrojů byly vloženy úpravy
noveCisloVerze: 1000
Automatické zvýšení verze pro noční buildy: 1000
Ostre:  zdroje byly zvyseny na 1000

C:\HeO_vyroba_distribuce\Zdroje\Stare\asseco\_Verze - kopie.inc
Aktualni cislo verze na SVN - HeO2: 0961 a HeO3: 0961
Aktualni cislo verze v INI - HeO2: 0961 a HeO3: 0961
Stare:  zdroje nebyly zvednuty manuálně
Stare:  do zdrojů byly vloženy úpravy
noveCisloVerze: 0962
Automatické zvýšení verze pro noční buildy: 0962
Stare:  zdroje byly zvyseny na 0962

C:\HeO_vyroba_distribuce\Zdroje\Freeze\asseco\_Verze - k

In [2]:
print("Start")
time.sleep(50)
print("Konec")

Start
Konec
